In [1]:
import glob
import numpy as np
from osgeo import gdal


In [10]:
in_dir = '/datalake/S2-L1C/31TFJ/2021/09/01/S2B_MSIL1C_20210901T102559_N0301_R108_T31TFJ_20210901T123557.SAFE'

# Search directory for desired bands
imgdir='/GRANULE/*/IM*/*'
red_file = glob.glob(in_dir + imgdir+'B04.jp2') # red band
nir_file = glob.glob(in_dir + imgdir+'B08.jp2') # nir band
print(red_file)

['/datalake/S2-L1C/31TFJ/2021/09/01/S2B_MSIL1C_20210901T102559_N0301_R108_T31TFJ_20210901T123557.SAFE/GRANULE/L1C_T31TFJ_A023443_20210901T103314/IMG_DATA/T31TFJ_20210901T102559_B04.jp2']


In [11]:
# Define a function to calculate NDVI using band arrays for red, NIR bands
def ndvi(red, nir):
 return ((nir - red)/(nir + red))

# Open each band using gdal
red_link = gdal.Open(red_file[0])
nir_link = gdal.Open(nir_file[0])

In [12]:
# read in each band as array and convert to float for calculations
red = red_link.ReadAsArray().astype(np.float)
nir = nir_link.ReadAsArray().astype(np.float)
red

/tmp/pbs.22252899.admin01/ipykernel_4870/1372501167.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  red = red_link.ReadAsArray().astype(np.float)
/tmp/pbs.22252899.admin01/ipykernel_4870/1372501167.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  nir = nir_link.ReadAsArray().astype(np.float)


array([[   0.,    0.,    0., ..., 1377., 1390., 1368.],
       [   0.,    0.,    0., ..., 1363., 1321., 1248.],
       [   0.,    0.,    0., ..., 1144., 1073., 1037.],
       ...,
       [ 677.,  661.,  650., ...,  622.,  642.,  627.],
       [ 642.,  652.,  650., ...,  646.,  638.,  570.],
       [ 608.,  601.,  594., ...,  656.,  606.,  566.]])

In [ ]:
# Call the ndvi() function on red, NIR bands
ndvi2 = ndvi(red, nir)

# Create output filename based on input name
outfile_name = red_file[0].split('_B')[0] + '_NDVI.tif'

x_pixels = ndvi2.shape[0] # number of pixels in x
y_pixels = ndvi2.shape[1] # number of pixels in y

# Set up output GeoTIFF
driver = gdal.GetDriverByName('GTiff')

# Create driver using output filename, x and y pixels, # of bands, and datatype
ndvi_data = driver.Create(outfile_name,x_pixels, y_pixels, 1,gdal.GDT_Float32)

# Set NDVI array as the 1 output raster band
ndvi_data.GetRasterBand(1).WriteArray(ndvi2)

# Setting up the coordinate reference system of the output GeoTIFF
geotrans=red_link.GetGeoTransform() # Grab input GeoTranform information
proj=red_link.GetProjection() # Grab projection information from input file

# now set GeoTransform parameters and projection on the output file
ndvi_data.SetGeoTransform(geotrans)
ndvi_data.SetProjection(proj)
ndvi_data.FlushCache()
ndvi_data=None